# Error fields
This notebook illustrates the error field computation using different techniques, including the so-called *Clever Poor Man's Error* (CPME) method.

In [ ]:
using DIVAnd
using PyPlot
if VERSION >= v"0.7.0-beta.0"
    using Dates
    using Statistics
else
    using Compat: @info, @warn, @debug
end
using Compat

## Data reading
Download the file (it not already done) and read it.

In [ ]:
varname = "Salinity"
filename = "data/WOD-Salinity-Provencal.nc"

if !isfile(filename)    
    download("https://b2drop.eudat.eu/s/UsF3RyU3xB1UM2o/download",filename)
else 
    @info "File already downloaded"
end

In [ ]:
obsval,obslon,obslat,obsdepth,obstime,obsid = loadobs(Float64,filename,"Salinity");

## Topography and grid definition
See the notebook on [bathymetry](06-topography.ipynb) for more explanations.

Define domain and resolution, create the domain.

In [ ]:
dx = dy = 0.125/2.
lonr = 2.5:dx:12.
latr = 42.3:dy:44.6

mask,(pm,pn),(xi,yi) = DIVAnd_rectdom(lonr,latr);

Download the bathymetry file and load it.

In [ ]:
bathname = "data/gebco_30sec_4.nc"
if !isfile(bathname)
    download("https://b2drop.eudat.eu/s/ACcxUEZZi6a4ziR/download",bathname)
else
    @info("Bathymetry file already downloaded")
end

In [ ]:
bx,by,b = load_bath(bathname,true,lonr,latr);

Create a land-sea mask based on the bathymetry.

In [ ]:
mask = falses(size(b,1),size(b,2))

for j = 1:size(b,2)
    for i = 1:size(b,1)
        mask[i,j] = b[i,j] >=1.0
    end
end

## Data selection for example

Cross validation, error calculations etc. assume independant data. Hence do not take high-resolution vertical profiles with all data but restrict yourself to specific small depth range. Here we limit outselves to August data at surface:

In [ ]:
sel = (obsdepth .< 1) .& (Dates.month.(obstime) .== 8)

obsval = obsval[sel]
obslon = obslon[sel]
obslat = obslat[sel]
obsdepth = obsdepth[sel]
obstime = obstime[sel]
obsid = obsid[sel];
@show (size(obsval))
checkobs((obslon,obslat,obsdepth,obstime),obsval,obsid)

### Analysis
Analysis parameters have been calibrated in the other notebook [13-processing-parameter-optimization example.ipynb](13-processing-parameter-optimization). 

**WARNING:** if the statistical parameters are incorrectly estimated, the error fields are meaningless and only provide an idea of data coverage.

The analysis parameters are:

In [ ]:
len=0.3
epsilon2=1.0;

Analysis `fi` using mean data as background.    
Structure `s` is stored for later use in error calculation.

In [ ]:
fi,s = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval.-mean(obsval),len,epsilon2);

Create a simple plot of the analysis

In [ ]:
pcolor(xi,yi,fi.+mean(obsval),vmin=37,vmax=38.5);
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))

## Exact error and approximations

Details can be found in the publication:

Approximate and Efficient Methods to Assess Error Fields in Spatial Gridding with Data Interpolating Variational Analysis (DIVA) Beckers, Jean-Marie; Barth, Alexander;  Troupin, Charles, Alvera-Azcarate, A.  *Journal of Atmospheric & Oceanic Technology* (2014), **31(2)**, 515-530     
https://orbi.uliege.be/handle/2268/161069      
https://journals.ametsoc.org/doi/abs/10.1175/JTECH-D-13-00130.1

In the 2D case you can try to calculate the exact error expression. This demands the computationally expensive evaluation of `diag(s.P)` accessible via the analysis returned structure `s`. This is only available with `DIVAndrun`.

In [ ]:
cmaperror = ColorMap("hot_r");
exerr,= statevector_unpack(s.sv,diag(s.P))
exerr[find(x -> x == false,mask)]=NaN

pcolor(xi,yi,exerr,vmin=0, vmax=1.5, cmap=cmaperror);
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))
title("Unscaled error");

Relative error by scaling with background variance `Berr` estimated using data with high errors

In [ ]:
epsilon2huge=1E6
fib,sb = DIVAndrun(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval,len,epsilon2huge);
Berr,= statevector_unpack(sb.sv,diag(sb.P))

In [ ]:
figure()
pcolor(xi,yi,exerr./Berr,vmin=0, vmax=1.5, cmap=cmaperror);
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))
title("Scaled error");

Now *clever poor man's error*.     
Usage as for `DIVAndrun`.

In [ ]:
cpme = DIVAnd_cpme(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval,len,epsilon2)

pcolor(xi,yi,cpme,vmin=0, vmax=1.5, cmap=cmaperror);
colorbar(orientation="horizontal")
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))
title("Clever poor man's error");

Do you see any difference between the exact and clever poor man's error ? 
## Difference between error fields
We also overlay the data positions.

In [ ]:
pcolor(xi,yi,cpme-exerr./Berr,vmin=-0.2, vmax=0.2, cmap="RdBu_r");
colorbar(orientation="horizontal")
plot(obslon, obslat, "k.", markersize=.5)
gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))
title("Error on error");

Another approximation to the error field: AEXERR

In [ ]:
myerr,bjmb,fa,sa = DIVAnd_aexerr(mask,(pm,pn),(xi,yi),(obslon,obslat),obsval,len,epsilon2)
if myerr==0
    error("No need to approximate error, use direct calculation")
else
    pcolor(xi,yi,myerr,vmin=0, vmax=1.5, cmap=cmaperror);
    colorbar(orientation="horizontal")
    gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))
    title("Unscaled almost exact error");
end

Scaled AXERR error

In [ ]:
if myerr==0
    error("No need to approximate error, use direct calculation")
else
    pcolor(xi,yi,myerr./bjmb,vmin=0, vmax=1.5, cmap=cmaperror);
    colorbar(orientation="horizontal")
    gca()[:set_aspect](1/cos(mean([ylim()...]) * pi/180))
    title("Scaled almost exact error");
end

# Exercise 
1. Modify the (L, $\epsilon^2$) parameters.
2. Re-run the analysis.
3. See how the error field behaves.

# Conclusion
In view of the uncertainties on statistical parameters (L, $\epsilon^2$), the *clever poor man's error* is generally a sufficient approximation for the error fields. This is the one implemented in the DIVAndgo high-level analysis function. 